In [11]:
import itemQuery as iq
import articleParse as ap
from timeit import default_timer as timer

In [12]:
%run modelInit.py

In [13]:
def getFeeds():
    """Google Sheets API Code.
    Pulls urls for all NFL Team RSS Feeds
    https://docs.google.com/spreadsheets/d/1DUBb9OG1A1Xs6v2PK9Oislz4384DSu2MzEbH8dK0ad4/edit#gid=0
    """
    credentials = get_credentials()
    http = credentials.authorize(httplib2.Http())
    discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                    'version=v4')
    service = discovery.build('sheets', 'v4', http=http,
                              discoveryServiceUrl=discoveryUrl)

    #specify sheetID and range
    spreadsheetId = '1DUBb9OG1A1Xs6v2PK9Oislz4384DSu2MzEbH8dK0ad4'
    rangeName = 'Sheet1!A2:E'
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheetId, range=rangeName).execute()
    values = result.get('values', [])

    if not values:
        print('No data found.')
    else:
        print('Done')

    return values

In [14]:
feeds = getFeeds()

Done


In [15]:
feeds

[['0',
  'Steelers',
  'News',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=News',
  'Steelers.com'],
 ['1',
  'Steelers',
  'Photos',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Photos',
  'Steelers.com'],
 ['2',
  'Steelers',
  'Videos',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Videos',
  'Steelers.com'],
 ['3',
  'Steelers',
  'Audio',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Audio',
  'Steelers.com'],
 ['4',
  'Steelers',
  'Espanol',
  'http://www.steelers.com/cda-web/rss-module.htm?tagName=Espa%C3%B1ol',
  'Steelers.com'],
 ['5',
  'Ravens',
  'All News',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News'],
 ['6',
  'Ravens',
  'Press Releases',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Press%20Releases'],
 ['7',
  'Ravens',
  'Transcripts',
  'http://www.baltimoreravens.com/cda-web/rss-module.htm?tagName=News%20-%20Transcripts'],
 ['8',
  'Ravens',
  'Late For Work'

In [6]:
demo = iq.recordsFromFeed('http://www.steelers.com/cda-web/rss-module.htm?tagName=News')

In [7]:
demo

[('Thu, 17 Aug 2017 22:45:00 GMT',
  'Offensive line answers the challenge',
  'http://www.steelers.com/news/article-1/Offensive-line-answers-the-challenge/98908728-8df5-46be-acd8-e8e40988b561',
  'Work despite depleted ranks pleases Tomlin.',
  'Mike Prisuta'),
 ('Thu, 17 Aug 2017 17:15:00 GMT',
  'Groundbreaking takes place for UPMC Field',
  'http://www.steelers.com/news/article-1/Groundbreaking-takes-place-for-UPMC-Field/da7cb5c0-d87d-4264-af2e-291a8c625d12',
  'The Steelers will be among those able to use the new field at Saint Vincent College.',
  'Teresa Varley'),
 ('Thu, 17 Aug 2017 13:30:00 GMT',
  'Roethlisberger likes what he sees',
  'http://www.steelers.com/news/article-1/Roethlisberger-likes-what-he-sees/73877e69-9600-48ef-a27c-fe3b8e35bd63',
  "Ben Roethlisberger is happy with the progress the offense has made, and Martavis Bryant's approach.",
  'Teresa Varley'),
 ('Thu, 17 Aug 2017 10:00:00 GMT',
  'Asked and Answered: Aug. 17',
  'http://www.steelers.com/news/asked-an

In [21]:
def feedFrame(feedRow):
    return [{'pubDate':record[0], 
             'team':feedRow[1], 
             'title':record[1], 
             'type':feedRow[2], 
             'link':record[2], 
             'discription':record[3], 
             'creator':record[4]
            } for record in iq.recordsFromFeed(feedRow[3])]
    

In [22]:
feedFrame(feeds[0])

[{'creator': 'Mike Prisuta',
  'discription': 'Work despite depleted ranks pleases Tomlin.',
  'link': 'http://www.steelers.com/news/article-1/Offensive-line-answers-the-challenge/98908728-8df5-46be-acd8-e8e40988b561',
  'pubDate': 'Thu, 17 Aug 2017 22:45:00 GMT',
  'team': 'Steelers',
  'title': 'Offensive line answers the challenge',
  'type': 'News'},
 {'creator': 'Teresa Varley',
  'discription': 'The Steelers will be among those able to use the new field at Saint Vincent College.',
  'link': 'http://www.steelers.com/news/article-1/Groundbreaking-takes-place-for-UPMC-Field/da7cb5c0-d87d-4264-af2e-291a8c625d12',
  'pubDate': 'Thu, 17 Aug 2017 17:15:00 GMT',
  'team': 'Steelers',
  'title': 'Groundbreaking takes place for UPMC Field',
  'type': 'News'},
 {'creator': 'Teresa Varley',
  'discription': "Ben Roethlisberger is happy with the progress the offense has made, and Martavis Bryant's approach.",
  'link': 'http://www.steelers.com/news/article-1/Roethlisberger-likes-what-he-sees/

In [23]:
data = []
for feedRow in feeds:
    if feedRow[3] == 'null':
        continue
    data.extend(feedFrame(feedRow))
    print('team: '+ feedRow[1] + ' ' + feedRow[0] )

team: Steelers 0
team: Steelers 1
team: Steelers 2
team: Steelers 3
team: Steelers 4
team: Ravens 5
team: Ravens 6
team: Ravens 7
team: Ravens 8
team: Ravens 9
team: Ravens 10
team: Ravens 11
team: Ravens 12
team: Ravens 13
team: Ravens 14
team: Ravens 15
team: Ravens 16
team: Ravens 17
team: Bengals 18
team: Bengals 19
team: Bengals 20
team: Bengals 21
team: Bengals 22
team: Browns 23
team: Browns 24
team: Browns 25
team: Browns 26
team: Packers 28
team: Packers 29
team: Packers 30
team: Packers 31
team: Packers 32
team: Packers 33
team: Vikings 34
team: Vikings 35
team: Vikings 36
team: Vikings 37
team: Lions 38
team: Lions 39
team: Lions 40
team: Lions 41
team: Lions 42
team: Texans 43
team: Texans 44
team: Titans 45
team: Titans 46
team: Titans 47
team: Titans 48
team: Titans 49
team: Titans 50
team: Titans 51
team: Titans 52
team: Jaguars 53
team: Jaguars 54
team: Jaguars 55
team: Colts 56
team: Colts 57
team: Colts 58
team: Saints 59
team: Saints 60
team: Saints 61
team: Saints 6

In [24]:
data

[{'creator': 'Mike Prisuta',
  'discription': 'Work despite depleted ranks pleases Tomlin.',
  'link': 'http://www.steelers.com/news/article-1/Offensive-line-answers-the-challenge/98908728-8df5-46be-acd8-e8e40988b561',
  'pubDate': 'Thu, 17 Aug 2017 22:45:00 GMT',
  'team': 'Steelers',
  'title': 'Offensive line answers the challenge',
  'type': 'News'},
 {'creator': 'Teresa Varley',
  'discription': 'The Steelers will be among those able to use the new field at Saint Vincent College.',
  'link': 'http://www.steelers.com/news/article-1/Groundbreaking-takes-place-for-UPMC-Field/da7cb5c0-d87d-4264-af2e-291a8c625d12',
  'pubDate': 'Thu, 17 Aug 2017 17:15:00 GMT',
  'team': 'Steelers',
  'title': 'Groundbreaking takes place for UPMC Field',
  'type': 'News'},
 {'creator': 'Teresa Varley',
  'discription': "Ben Roethlisberger is happy with the progress the offense has made, and Martavis Bryant's approach.",
  'link': 'http://www.steelers.com/news/article-1/Roethlisberger-likes-what-he-sees/

In [25]:
len(data)

2102

In [26]:
import pandas as pd

In [27]:
df = pd.DataFrame(data)

In [28]:
df

,creator,discription,link,pubDate,team,title,type
0,Mike Prisuta,Work despite depleted ranks pleases Tomlin.,http://www.steelers.com/news/article-1/Offensi...,"Thu, 17 Aug 2017 22:45:00 GMT",Steelers,Offensive line answers the challenge,News
1,Teresa Varley,The Steelers will be among those able to use t...,http://www.steelers.com/news/article-1/Groundb...,"Thu, 17 Aug 2017 17:15:00 GMT",Steelers,Groundbreaking takes place for UPMC Field,News
2,Teresa Varley,Ben Roethlisberger is happy with the progress ...,http://www.steelers.com/news/article-1/Roethli...,"Thu, 17 Aug 2017 13:30:00 GMT",Steelers,Roethlisberger likes what he sees,News
3,Bob Labriola,Another installment of Bob Labriola answering ...,http://www.steelers.com/news/asked-and-answere...,"Thu, 17 Aug 2017 10:00:00 GMT",Steelers,Asked and Answered: Aug. 17,News
4,Mike Prisuta,Tomlin pays close attention to receiver's retu...,http://www.steelers.com/news/article-1/Coates-...,"Wed, 16 Aug 2017 23:00:49 GMT",Steelers,Coates under the microscope,News
5,Bob Labriola,"In 2017's second depth chart, Bryant is a star...",http://www.steelers.com/news/article-1/Bryant-...,"Wed, 16 Aug 2017 15:39:02 GMT",Steelers,Bryant installed as a starter at WR,News
6,Teresa Varley,Sammie Coates is looking forward to getting ba...,http://www.steelers.com/news/article-1/Steeler...,"Wed, 16 Aug 2017 14:00:00 GMT",Steelers,Steelers activate Coates from Active/PUP list,News
7,Teresa Varley,Alan Faneca is working with the Steelers offen...,http://www.steelers.com/news/article-1/Faneca-...,"Wed, 16 Aug 2017 10:55:00 GMT",Steelers,Faneca passing on the knowledge,News
8,Teresa Varley,L.J. Fort is inspired by the SEAL's principles...,http://www.steelers.com/news/article-1/Navy-SE...,"Tue, 15 Aug 2017 16:00:00 GMT",Steelers,Navy SEAL's discipline inspires Fort,News
9,Bob Labriola,Another installment of Bob Labriola answering ...,http://www.steelers.com/news/asked-and-answere...,"Tue, 15 Aug 2017 10:00:00 GMT",Steelers,Asked and Answered: Aug. 15,News
